# Time Series Data Extraction for ARDS Cohort

This notebook extracts all time series variables from the analysis_dataset_schema.png for our ARDS cohort:
- Demographics and static variables
- Ventilation parameters (PEEP, FiO2, SpO2, etc.)
- Neuromuscular blockade agents (doses)
- Proning events
- Outcomes (mortality, LOS, extubation)
- All timestamped relative to ARDS onset

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Define paths
MIMIC_PATH = '/Users/kavenchhikara/Desktop/CLIF/MIMIC-IV-3.1/physionet.org/files'
DATA_PATH = '/Users/kavenchhikara/Desktop/projects/SCCM/SCCM-Team2/ards_analysis/data'
MAPPING_PATH = '/Users/kavenchhikara/Desktop/projects/SCCM/SCCM-Team2/mimic_mapping.xlsx'

print(f"Analysis start time: {datetime.now()}")

## Step 1: Load ARDS Cohort and Mapping File

In [ ]:
# Load ARDS cohort
ards_cohort = pd.read_parquet(f'{DATA_PATH}/ards_cohort.parquet')
print(f"ARDS cohort loaded: {len(ards_cohort):,} patients")

# Convert datetime columns
datetime_cols = ['admission_dttm', 'discharge_dttm', 'ards_onset_time', 'bilateral_infiltrates_time']
for col in datetime_cols:
    ards_cohort[col] = pd.to_datetime(ards_cohort[col])

# Load MIMIC mapping
mimic_mapping = pd.read_excel(MAPPING_PATH)
print(f"\nMIMIC mapping loaded: {len(mimic_mapping)} variables")

In [ ]:
# Display mapping for key variables
print("Neuromuscular blockade agents:")
nmb_agents = ['cisatracurium', 'vecuronium', 'rocuronium', 'pancuronium', 'atracurium']
nmb_mapping = mimic_mapping[mimic_mapping['variable'].isin(nmb_agents)]
print(nmb_mapping[['variable', 'itemid', 'label', 'unitname']])

print("\nVentilation parameters:")
vent_vars = ['fio2_set', 'peep_set', 'spo2', 'mode_name', 'device_name']
vent_mapping = mimic_mapping[mimic_mapping['variable'].isin(vent_vars)]
print(vent_mapping[['variable', 'itemid', 'label']])

print("\nPosition/Proning:")
position_mapping = mimic_mapping[mimic_mapping['variable'] == 'position']
print(position_mapping[['variable', 'itemid', 'label', 'value_instances']])

## Step 2: Extract Demographics and Static Variables

In [ ]:
# Load ICU stays for APACHE scores
icustays = pd.read_csv(f'{MIMIC_PATH}/mimiciv/3.1/icu/icustays.csv.gz')
first_day_apache = pd.read_csv(f'{MIMIC_PATH}/mimiciv/3.1/icu/first_day_apache_ii.csv.gz')

# Merge APACHE scores
ards_with_apache = ards_cohort.merge(
    icustays[['hadm_id', 'stay_id']], 
    on='hadm_id', 
    how='left'
).merge(
    first_day_apache[['stay_id', 'apache_ii']], 
    on='stay_id', 
    how='left'
)

# Get first ICU stay APACHE score for each admission
apache_scores = ards_with_apache.groupby('hadm_id')['apache_ii'].first().reset_index()
apache_scores.rename(columns={'apache_ii': 'APACHE'}, inplace=True)

print(f"APACHE scores available for {apache_scores['APACHE'].notna().sum()} patients")

In [ ]:
# Extract height and weight
print("Extracting height and weight...")

# Get height/weight itemids from mapping
height_itemids = mimic_mapping[mimic_mapping['variable'] == 'height_cm']['itemid'].dropna().astype(int).tolist()
weight_itemids = mimic_mapping[mimic_mapping['variable'].isin(['weight_kg', 'weight_lbs'])]['itemid'].dropna().astype(int).tolist()

# Extract from chartevents
height_weight_data = []

# Get ICU stays for our cohort
cohort_stays = icustays[icustays['hadm_id'].isin(ards_cohort['hadm_id'])]['stay_id'].tolist()

chunk_size = 1000000
for i, chunk in enumerate(pd.read_csv(f'{MIMIC_PATH}/mimiciv/3.1/icu/chartevents.csv.gz', 
                                     chunksize=chunk_size,
                                     usecols=['stay_id', 'itemid', 'valuenum', 'valueuom'])):
    # Filter for our cohort and height/weight items
    hw_chunk = chunk[
        (chunk['stay_id'].isin(cohort_stays)) &
        (chunk['itemid'].isin(height_itemids + weight_itemids))
    ]
    
    if len(hw_chunk) > 0:
        height_weight_data.append(hw_chunk)
    
    if i % 10 == 0:
        print(f"Processed {(i+1)*chunk_size:,} rows...")

# Combine and process
if height_weight_data:
    hw_df = pd.concat(height_weight_data, ignore_index=True)
    hw_df = hw_df.merge(icustays[['stay_id', 'hadm_id']], on='stay_id', how='left')
    
    # Process height (convert to cm if needed)
    height_df = hw_df[hw_df['itemid'].isin(height_itemids)].groupby('hadm_id')['valuenum'].first().reset_index()
    height_df.rename(columns={'valuenum': 'height_cm'}, inplace=True)
    
    # Process weight (convert to kg if needed)
    weight_df = hw_df[hw_df['itemid'].isin(weight_itemids)].copy()
    # Convert lbs to kg where applicable
    lbs_mask = weight_df['valueuom'] == 'lbs'
    weight_df.loc[lbs_mask, 'valuenum'] = weight_df.loc[lbs_mask, 'valuenum'] * 0.453592
    weight_df = weight_df.groupby('hadm_id')['valuenum'].first().reset_index()
    weight_df.rename(columns={'valuenum': 'weight_kg'}, inplace=True)
    
    print(f"\nHeight data found for {len(height_df)} patients")
    print(f"Weight data found for {len(weight_df)} patients")

## Step 3: Extract Time Series Ventilation Parameters

In [ ]:
# Load previously extracted ventilation data
vent_data = pd.read_parquet(f'{DATA_PATH}/ventilation_parameters.parquet')
vent_data['charttime'] = pd.to_datetime(vent_data['charttime'])

# Get additional ventilation parameters from mapping
additional_vent_itemids = {
    'fio2_set': mimic_mapping[mimic_mapping['variable'] == 'fio2_set']['itemid'].dropna().astype(int).tolist(),
    'peep_set': mimic_mapping[mimic_mapping['variable'] == 'peep_set']['itemid'].dropna().astype(int).tolist(),
    'spo2': mimic_mapping[mimic_mapping['variable'] == 'spo2']['itemid'].dropna().astype(int).tolist(),
    'device_name': mimic_mapping[mimic_mapping['variable'] == 'device_name']['itemid'].dropna().astype(int).tolist(),
    'ecmo_flag': mimic_mapping[mimic_mapping['variable'] == 'device_name_ecmo']['itemid'].dropna().astype(int).tolist()
}

print("Extracting additional ventilation parameters...")
additional_vent_data = []

all_additional_itemids = [item for items in additional_vent_itemids.values() for item in items]

for i, chunk in enumerate(pd.read_csv(f'{MIMIC_PATH}/mimiciv/3.1/icu/chartevents.csv.gz', 
                                     chunksize=chunk_size)):
    # Filter for our cohort and items
    chunk_filtered = chunk[
        (chunk['stay_id'].isin(cohort_stays)) &
        (chunk['itemid'].isin(all_additional_itemids))
    ]
    
    if len(chunk_filtered) > 0:
        additional_vent_data.append(chunk_filtered)
    
    if i % 10 == 0:
        print(f"Processed {(i+1)*chunk_size:,} rows...")

# Process additional ventilation data
if additional_vent_data:
    add_vent_df = pd.concat(additional_vent_data, ignore_index=True)
    add_vent_df = add_vent_df.merge(icustays[['stay_id', 'hadm_id']], on='stay_id', how='left')
    add_vent_df['charttime'] = pd.to_datetime(add_vent_df['charttime'])

## Step 4: Extract Neuromuscular Blockade Administration

In [ ]:
# Get NMB itemids from mapping
nmb_itemids = {}
for agent in nmb_agents:
    itemids = mimic_mapping[mimic_mapping['variable'] == agent]['itemid'].dropna().astype(int).tolist()
    if itemids:
        nmb_itemids[agent] = itemids

print("Neuromuscular blockade itemids:")
for agent, ids in nmb_itemids.items():
    print(f"{agent}: {ids}")

# Extract NMB administration from inputevents_mv
print("\nExtracting NMB administration data...")
all_nmb_itemids = [item for items in nmb_itemids.values() for item in items]

nmb_data = []
for i, chunk in enumerate(pd.read_csv(f'{MIMIC_PATH}/mimiciv/3.1/icu/inputevents.csv.gz', 
                                     chunksize=chunk_size)):
    # Filter for NMB medications
    nmb_chunk = chunk[
        (chunk['stay_id'].isin(cohort_stays)) &
        (chunk['itemid'].isin(all_nmb_itemids))
    ]
    
    if len(nmb_chunk) > 0:
        nmb_data.append(nmb_chunk)
    
    if i % 10 == 0:
        print(f"Processed {(i+1)*chunk_size:,} rows...")

# Process NMB data
if nmb_data:
    nmb_df = pd.concat(nmb_data, ignore_index=True)
    nmb_df = nmb_df.merge(icustays[['stay_id', 'hadm_id']], on='stay_id', how='left')
    nmb_df['starttime'] = pd.to_datetime(nmb_df['starttime'])
    nmb_df['endtime'] = pd.to_datetime(nmb_df['endtime'])
    
    # Map itemids back to drug names
    itemid_to_drug = {}
    for drug, itemids in nmb_itemids.items():
        for itemid in itemids:
            itemid_to_drug[itemid] = drug
    
    nmb_df['drug_name'] = nmb_df['itemid'].map(itemid_to_drug)
    
    print(f"\nNMB administrations found: {len(nmb_df):,}")
    print("\nNMB drugs administered:")
    print(nmb_df['drug_name'].value_counts())

## Step 5: Extract Proning Events

In [ ]:
# Get position itemids from mapping
position_itemids = mimic_mapping[mimic_mapping['variable'] == 'position']['itemid'].dropna().astype(int).tolist()
print(f"Position itemids: {position_itemids}")

# Extract position data
print("\nExtracting position/proning data...")
position_data = []

for i, chunk in enumerate(pd.read_csv(f'{MIMIC_PATH}/mimiciv/3.1/icu/chartevents.csv.gz', 
                                     chunksize=chunk_size,
                                     usecols=['stay_id', 'itemid', 'charttime', 'value'])):
    # Filter for position items
    pos_chunk = chunk[
        (chunk['stay_id'].isin(cohort_stays)) &
        (chunk['itemid'].isin(position_itemids))
    ]
    
    if len(pos_chunk) > 0:
        position_data.append(pos_chunk)
    
    if i % 10 == 0:
        print(f"Processed {(i+1)*chunk_size:,} rows...")

# Process position data
if position_data:
    position_df = pd.concat(position_data, ignore_index=True)
    position_df = position_df.merge(icustays[['stay_id', 'hadm_id']], on='stay_id', how='left')
    position_df['charttime'] = pd.to_datetime(position_df['charttime'])
    
    # Identify proning events
    prone_keywords = ['prone', 'proning', 'pronation']
    position_df['is_prone'] = position_df['value'].str.lower().str.contains('|'.join(prone_keywords), na=False)
    
    prone_events = position_df[position_df['is_prone']]
    print(f"\nProne position events found: {len(prone_events):,}")
    print(f"Patients with proning: {prone_events['hadm_id'].nunique()}")

## Step 6: Extract Tracheostomy Events

In [ ]:
# Load procedures to identify tracheostomy
procedures = pd.read_csv(f'{MIMIC_PATH}/mimiciv/3.1/icu/procedureevents.csv.gz')

# Get tracheostomy itemids from mapping
trach_itemids = mimic_mapping[mimic_mapping['variable'] == 'tracheostomy']['itemid'].dropna().astype(int).tolist()
print(f"Tracheostomy itemids: {trach_itemids}")

# Filter for tracheostomy procedures
trach_procedures = procedures[
    (procedures['stay_id'].isin(cohort_stays)) &
    (procedures['itemid'].isin(trach_itemids))
]

if len(trach_procedures) > 0:
    trach_procedures = trach_procedures.merge(icustays[['stay_id', 'hadm_id']], on='stay_id', how='left')
    trach_procedures['starttime'] = pd.to_datetime(trach_procedures['starttime'])
    
    # Get first tracheostomy for each admission
    first_trach = trach_procedures.groupby('hadm_id')['starttime'].min().reset_index()
    first_trach.rename(columns={'starttime': 'tracheostomy_time'}, inplace=True)
    
    print(f"\nPatients with tracheostomy: {len(first_trach)}")
else:
    first_trach = pd.DataFrame(columns=['hadm_id', 'tracheostomy_time'])
    print("\nNo tracheostomy procedures found in cohort")

## Step 7: Create Time Series Dataset

In [ ]:
# Function to create hourly time series
def create_time_series_data(hadm_id, ards_onset_time, admission_time, discharge_time):
    """
    Create hourly time series for a patient relative to ARDS onset
    """
    # Create hourly timestamps from admission to discharge
    hours = pd.date_range(start=admission_time, end=discharge_time, freq='H')
    
    # Create base dataframe
    ts_data = pd.DataFrame({
        'hadm_id': hadm_id,
        'recorded_dttm': hours,
        'time_from_ARDS_onset': (hours - ards_onset_time).total_seconds() / 3600  # hours from ARDS onset
    })
    
    return ts_data

# Create time series framework for all ARDS patients
print("Creating time series framework...")
all_ts_data = []

for idx, row in ards_cohort.iterrows():
    ts_data = create_time_series_data(
        row['hadm_id'],
        row['ards_onset_time'],
        row['admission_dttm'],
        row['discharge_dttm']
    )
    all_ts_data.append(ts_data)
    
    if idx % 100 == 0:
        print(f"Processed {idx} patients...")

# Combine all time series
time_series_df = pd.concat(all_ts_data, ignore_index=True)
print(f"\nTotal time series rows created: {len(time_series_df):,}")

In [ ]:
# Add static variables
print("Adding static variables...")

# Merge demographics from ARDS cohort
static_vars = ards_cohort[[
    'hadm_id', 'subject_id', 'hospital_id', 'hospitalization_id',
    'admission_dttm', 'discharge_dttm', 'age_at_admission', 
    'gender', 'ethnicity', 'ards_severity'
]].copy()

# Rename columns to match schema
static_vars.rename(columns={
    'gender': 'sex',
    'hadm_id': 'hospitalization_id_temp'
}, inplace=True)
static_vars['hospital_id'] = 'BIDMC'  # All MIMIC data is from BIDMC
static_vars['patient_id'] = static_vars['subject_id']
static_vars['hospitalization_id'] = static_vars['hospitalization_id_temp']

# Add APACHE scores
static_vars = static_vars.merge(apache_scores, left_on='hospitalization_id_temp', right_on='hadm_id', how='left')

# Add height and weight
if 'height_df' in locals():
    static_vars = static_vars.merge(height_df, left_on='hospitalization_id_temp', right_on='hadm_id', how='left')
if 'weight_df' in locals():
    static_vars = static_vars.merge(weight_df, left_on='hospitalization_id_temp', right_on='hadm_id', how='left')

# Merge with time series
time_series_df = time_series_df.merge(
    static_vars.drop(columns=['hospitalization_id_temp', 'hadm_id_x', 'hadm_id_y'], errors='ignore'),
    left_on='hadm_id',
    right_on='hospitalization_id',
    how='left'
)

In [ ]:
# Add time-varying ventilation parameters
print("\nAdding time-varying ventilation parameters...")

# Function to get nearest value within time window
def get_nearest_value(ts_row, value_df, value_col, time_col='charttime', window_hours=1):
    """
    Get nearest value within specified time window
    """
    hadm_id = ts_row['hadm_id']
    target_time = ts_row['recorded_dttm']
    
    # Filter to same admission
    hadm_values = value_df[value_df['hadm_id'] == hadm_id]
    
    if len(hadm_values) == 0:
        return np.nan
    
    # Calculate time differences
    time_diff = abs((hadm_values[time_col] - target_time).dt.total_seconds() / 3600)
    
    # Find values within window
    within_window = time_diff <= window_hours
    
    if within_window.sum() == 0:
        return np.nan
    
    # Return value with minimum time difference
    nearest_idx = time_diff[within_window].idxmin()
    return hadm_values.loc[nearest_idx, value_col]

# Add ventilation parameters (sample for first 1000 rows due to computational intensity)
sample_size = min(1000, len(time_series_df))
print(f"Processing sample of {sample_size} rows for demonstration...")

# For full dataset, you would remove the .head(sample_size)
for param in ['peep', 'fio2', 'pao2', 'pf_ratio']:
    if param in vent_data.columns:
        print(f"Adding {param}...")
        time_series_df.loc[:sample_size-1, param] = time_series_df.head(sample_size).apply(
            lambda x: get_nearest_value(x, vent_data, param), axis=1
        )

In [ ]:
# Add NMB doses
print("\nCalculating NMB doses...")

if 'nmb_df' in locals() and len(nmb_df) > 0:
    # Calculate hourly doses for each NMB agent
    for drug in nmb_agents:
        drug_data = nmb_df[nmb_df['drug_name'] == drug]
        
        if len(drug_data) > 0:
            # Initialize dose column
            time_series_df[f'{drug}_dose'] = 0.0
            
            # For each administration, calculate hourly dose
            for _, admin in drug_data.iterrows():
                # Find rows within administration period
                mask = (
                    (time_series_df['hadm_id'] == admin['hadm_id']) &
                    (time_series_df['recorded_dttm'] >= admin['starttime']) &
                    (time_series_df['recorded_dttm'] <= admin['endtime'])
                )
                
                if mask.sum() > 0:
                    # Calculate dose rate (amount/hour)
                    duration_hours = (admin['endtime'] - admin['starttime']).total_seconds() / 3600
                    if duration_hours > 0 and pd.notna(admin['amount']):
                        hourly_dose = admin['amount'] / duration_hours
                        
                        # Apply dose to matching rows
                        time_series_df.loc[mask, f'{drug}_dose'] += hourly_dose
else:
    # Initialize NMB dose columns as 0
    for drug in nmb_agents:
        time_series_df[f'{drug}_dose'] = 0.0

In [ ]:
# Add proning flag
print("\nAdding proning flags...")

time_series_df['prone_flag'] = 0

if 'prone_events' in locals() and len(prone_events) > 0:
    # For each prone event, mark the time period
    for hadm_id in prone_events['hadm_id'].unique():
        hadm_prone = prone_events[prone_events['hadm_id'] == hadm_id].sort_values('charttime')
        
        # Mark hours when patient was prone
        for _, event in hadm_prone.iterrows():
            # Mark 4 hours around each prone documentation (assuming prone sessions last ~4 hours)
            mask = (
                (time_series_df['hadm_id'] == hadm_id) &
                (time_series_df['recorded_dttm'] >= event['charttime'] - pd.Timedelta(hours=2)) &
                (time_series_df['recorded_dttm'] <= event['charttime'] + pd.Timedelta(hours=2))
            )
            time_series_df.loc[mask, 'prone_flag'] = 1

In [ ]:
# Add outcome variables
print("\nAdding outcome variables...")

# Hospital mortality (from ARDS cohort)
mortality_map = ards_cohort.set_index('hadm_id')['hospital_expire_flag'].to_dict()
time_series_df['hospital_mortality'] = time_series_df['hadm_id'].map(mortality_map)

# Disposition category
disposition_map = ards_cohort.set_index('hadm_id')['discharge_location'].to_dict()
time_series_df['disposition_category'] = time_series_df['hadm_id'].map(disposition_map)

# Categorize disposition
def categorize_disposition(location):
    if pd.isna(location):
        return 'Unknown'
    location_lower = location.lower()
    if 'expired' in location_lower or 'died' in location_lower:
        return 'Expired'
    elif 'hospice' in location_lower:
        return 'Hospice'
    elif 'home' in location_lower:
        return 'Home'
    elif 'snf' in location_lower or 'skilled' in location_lower:
        return 'Facility'
    elif 'rehab' in location_lower:
        return 'Facility'
    else:
        return 'Transfer to another facility'

time_series_df['disposition_category'] = time_series_df['disposition_category'].apply(categorize_disposition)

In [ ]:
# Add new_tracheostomy flag
print("\nAdding tracheostomy flags...")

time_series_df['new_tracheostomy'] = 0

if len(first_trach) > 0:
    # Mark new tracheostomy after it occurs
    for _, trach in first_trach.iterrows():
        mask = (
            (time_series_df['hadm_id'] == trach['hadm_id']) &
            (time_series_df['recorded_dttm'] >= trach['tracheostomy_time'])
        )
        time_series_df.loc[mask, 'new_tracheostomy'] = 1

## Step 8: Final Dataset Preparation

In [ ]:
# Select and order columns according to schema
schema_columns = [
    'hospital_id', 'patient_id', 'hospitalization_id', 'recorded_dttm',
    'time_from_ARDS_onset', 'APACHE', 'sex', 'age_at_admission', 'ethnicity',
    'disposition_category', 'respiratory_device', 'ecmo_flag',
    'pao2', 'fio2_set', 'lmp_set', 'spo2', 'peep',
    'height_cm', 'weight_kg',
    'cisatracurium_dose', 'vecuronium_dose', 'rocuronium_dose',
    'atracurium_dose', 'pancuronium_dose',
    'prone_flag', 'new_tracheostomy'
]

# Map column names
column_mapping = {
    'fio2': 'fio2_set',
    'hadm_id': 'hospitalization_id'
}

# Apply mapping
time_series_df.rename(columns=column_mapping, inplace=True)

# Add missing columns with default values
for col in schema_columns:
    if col not in time_series_df.columns:
        if col == 'respiratory_device':
            time_series_df[col] = 'Vent'  # Assume ventilated for ARDS patients
        elif col == 'ecmo_flag':
            time_series_df[col] = 0
        elif col == 'lmp_set':
            time_series_df[col] = np.nan
        else:
            time_series_df[col] = np.nan

# Select columns in schema order (only those that exist)
final_columns = [col for col in schema_columns if col in time_series_df.columns]
final_time_series = time_series_df[final_columns].copy()

print("Final dataset shape:", final_time_series.shape)
print("\nColumns in final dataset:")
print(final_columns)

## Step 9: Save Final Dataset

In [ ]:
# Save final time series dataset
output_file = f'{DATA_PATH}/ards_time_series_analysis.parquet'
final_time_series.to_parquet(output_file, index=False)
print(f"\nTime series dataset saved to: {output_file}")
print(f"File size: {os.path.getsize(output_file) / 1024 / 1024:.1f} MB")

# Summary statistics
print("\n=== DATASET SUMMARY ===")
print(f"Total rows: {len(final_time_series):,}")
print(f"Unique patients: {final_time_series['patient_id'].nunique():,}")
print(f"Unique hospitalizations: {final_time_series['hospitalization_id'].nunique():,}")
print(f"\nTime range relative to ARDS onset:")
print(f"Min: {final_time_series['time_from_ARDS_onset'].min():.1f} hours")
print(f"Max: {final_time_series['time_from_ARDS_onset'].max():.1f} hours")

print("\nProne positioning:")
prone_patients = final_time_series[final_time_series['prone_flag'] == 1]['hospitalization_id'].nunique()
print(f"Patients with proning: {prone_patients} ({prone_patients/final_time_series['hospitalization_id'].nunique()*100:.1f}%)")

print("\nNeuromuscular blockade:")
for drug in nmb_agents:
    drug_col = f'{drug}_dose'
    if drug_col in final_time_series.columns:
        drug_patients = final_time_series[final_time_series[drug_col] > 0]['hospitalization_id'].nunique()
        print(f"{drug}: {drug_patients} patients")

print(f"\nAnalysis completed at: {datetime.now()}")

## Next Steps

The time series dataset is now ready for analysis:
- Each row represents one hour of patient data
- All times are relative to ARDS onset (time=0)
- Includes all variables from the schema
- Ready for statistical modeling and outcome analysis

Note: For production use, remove the sampling limitation in Step 7 to process all rows.